In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def NSE(observed, simulated):

    mask = np.logical_or(np.isnan(observed), np.isnan(simulated))

    observed_masked = np.ma.masked_array(observed, mask)
    simulated_masked = np.ma.masked_array(simulated, mask)
    
    numerator = np.sum((observed_masked - simulated_masked) ** 2)
    denominator = np.sum((observed_masked - np.mean(observed_masked)) ** 2)

    return 1 - numerator / denominator

def KGE(observed, simulated):

    mask = np.logical_or(np.isnan(observed), np.isnan(simulated))

    observed_masked = np.ma.masked_array(observed, mask)
    simulated_masked = np.ma.masked_array(simulated, mask)
    
    r = np.ma.corrcoef(observed_masked, simulated_masked)[0, 1]
    alpha = np.mean(simulated_masked) / np.mean(observed_masked)
    beta = np.std(simulated_masked) / np.std(observed_masked)
    
    return 1 - np.sqrt((r - 1) ** 2 + (alpha - 1) ** 2 + (beta - 1) ** 2)

In [ ]:
def score_dframe(period):

    testdata = pd.read_csv(f"data-{period}.csv")
    preddata = pd.read_csv(f"data-corrected-{period}.csv")
    
    code = testdata["code"].unique()

    scores = {"code": [], "method": [], "nse": [], "kge": []}

    for i, c in enumerate(code):
        pred = (preddata[preddata.code==code[i]]["bias"] + preddata[preddata.code==code[i]]["discharge_sim"]).to_numpy()
        true = (testdata[testdata.code==code[i]]["bias"] + testdata[testdata.code==code[i]]["discharge_sim"]).to_numpy()
        sim = testdata[testdata.code==code[i]]["discharge_sim"].to_numpy()

        scores["code"].append(c) 
        scores["method"].append("classic")
        scores["nse"].append(NSE(true, sim))
        scores["kge"].append(KGE(true, sim))

        scores["code"].append(c) 
        scores["method"].append("lstm")
        scores["nse"].append(NSE(true, pred))
        scores["kge"].append(KGE(true, pred))

    return pd.DataFrame(scores)

In [ ]:
df1 = score_dframe("P1")
ax = df1.boxplot(column=["nse", "kge"], by="method")

In [ ]:
df2 = score_dframe("P2")
ax = df2.boxplot(column=["nse", "kge"], by="method")